Za zagon naslednjih zvezkov, če tega še niste storili, morate uvesti model, ki uporablja `text-embedding-ada-002` kot osnovni model, in nastaviti ime uvajanja v datoteki .env kot `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Nato bomo naložili indeks vdelav v podatkovni okvir Pandas. Indeks vdelav je shranjen v JSON datoteki z imenom `embedding_index_3m.json`. Indeks vdelav vsebuje vdelave za vsak YouTube prepis do poznega oktobra 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Naslednji korak je ustvariti funkcijo z imenom `get_videos`, ki bo poiskala po indeksu vdelav za dano poizvedbo. Funkcija bo vrnila 5 videoposnetkov, ki so najbolj podobni poizvedbi. Funkcija deluje tako:

1. Najprej se ustvari kopija indeksa vdelav.
2. Nato se za poizvedbo izračuna vdelava z uporabo OpenAI Embedding API.
3. Nato se v indeksu vdelav ustvari nov stolpec z imenom `similarity`. Stolpec `similarity` vsebuje kosinusno podobnost med vdelavo poizvedbe in vdelavo za vsak segment videoposnetka.
4. Nato se indeks vdelav filtrira po stolpcu `similarity`. Indeks vdelav se filtrira tako, da vključuje le videoposnetke, ki imajo kosinusno podobnost večjo ali enako 0,75.
5. Na koncu se indeks vdelav razvrsti po stolpcu `similarity` in vrne se najboljših 5 videoposnetkov.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ta funkcija je zelo preprosta, saj samo izpiše rezultate iskalne poizvedbe.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najprej se Embedding Index naloži v Pandas Dataframe.
2. Nato se uporabnika pozove, da vnese poizvedbo.
3. Nato se pokliče funkcija `get_videos`, ki v Embedding Index poišče rezultate za poizvedbo.
4. Na koncu se pokliče funkcija `display_results`, ki uporabniku prikaže rezultate.
5. Uporabnika se nato znova pozove, da vnese novo poizvedbo. Ta postopek se ponavlja, dokler uporabnik ne vnese `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.sl.png)

Pozvani boste, da vnesete poizvedbo. Vnesite poizvedbo in pritisnite enter. Aplikacija vam bo vrnila seznam videoposnetkov, ki so povezani z vašo poizvedbo. Prav tako boste prejeli povezavo do mesta v videoposnetku, kjer se nahaja odgovor na vaše vprašanje.

Tukaj je nekaj primerov poizvedb, ki jih lahko preizkusite:

- Kaj je Azure Machine Learning?
- Kako delujejo konvolucijske nevronske mreže?
- Kaj je nevronska mreža?
- Ali lahko uporabljam Jupyter Notebooks z Azure Machine Learning?
- Kaj je ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve AI prevajanja [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatski prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem maternem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
